In [3]:
import pandas as pd

In [54]:
df = pd.read_csv('../Data/quadruplet/model_inference_300_data.csv')
df.head(1)

,original_id,content,clean_tweet,final_sentiment,labels,quadruplet,spam,sentiment_label,model_prediction
0,1.640000e+18,@tokopedia min aku udah bayar tapi kenapa diba...,min aku udah bayar tapi kenapa dibatalin pesan...,negative,payment; produk;,"(pesananku, udah bayar tapi kenapa dibatalin, ...",NaN,sentiment,"(pesananku, sudah bayar tapi kenapa dibatalin,..."


# Utils

In [5]:
import re

In [41]:
def extract_quadruplet(sequence):
    extractions = []
    # find all matching quadruplet with (); pattern
    quadruplets = re.findall("\(.*?\)", sequence)
    for quadruplet in quadruplets:
        # Remove the in the start "("  and at the end ")".
        quadruplet = quadruplet[1:-1]
        try:
            aspect_term, opinion_term, sentiment, aspect_category = quadruplet.split(', ')
        except ValueError:
            aspect_term, opinion_term, sentiment, aspect_category = '', '', '', ''
        aspect_term = aspect_term.strip().lower()
        opinion_term = opinion_term.strip().lower()
        sentiment = sentiment.strip().lower()
        aspect_category = aspect_category.strip().lower()
        extractions.append((aspect_term, opinion_term, sentiment, aspect_category)) 
    return extractions

# Evaluator

In [52]:
import re

In [53]:
class Evaluator:
    def __init__(self):
        # == Metrics ==
        self.precision_fn = lambda n_tp, n_pred: float(n_tp) / float(n_pred) if n_pred != 0 else 0
        self.recall_fn = lambda n_tp, n_gold: float(n_tp) / float(n_gold) if n_gold != 0 else 0
        self.f1_fn = (
            lambda precision, recall: (2 * precision * recall) / (precision + recall)
            if precision != 0 or recall != 0
            else 0
        )
    def score(self, pred, gold):
        assert len(pred) == len(gold)
        n_tp, n_gold, n_pred = 0, 0, 0

        for i in range(len(pred)):
            n_gold += len(gold[i])
            n_pred += len(pred[i])

            for t in pred[i]:
                if t in gold[i]:
                    n_tp += 1
                

        precision = self.precision_fn(n_tp, n_pred)
        recall = self.recall_fn(n_tp, n_gold)
        f1 = self.f1_fn(precision, recall)
        return {"precision": precision, "recall": recall, "f1": f1}
    
    # == Evaluation ==
    def evaluate(self, pred_seqs, gold_seqs):
        assert len(pred_seqs) == len(gold_seqs)
        num_samples = len(gold_seqs)

        all_labels, all_preds = [], []

        for i in range(num_samples):
            gold_list = extract_quadruplet(gold_seqs[i])
            pred_list = extract_quadruplet(pred_seqs[i])

            all_labels.append(gold_list)
            all_preds.append(pred_list)

        raw_scores = self.score(all_preds, all_labels)
        return raw_scores, all_labels, all_preds

In [55]:
evaluator = Evaluator()
raw_scores, all_labels, all_preds = evaluator.evaluate(df['model_prediction'].astype('str'), df['quadruplet'])

In [56]:
raw_scores

{'precision': 0.3333333333333333,
 'recall': 0.3333333333333333,
 'f1': 0.3333333333333333}

# Preprocess

In [14]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import preprocessor as p
from src.slang_word import SLANG_WORDS
#gk kepake aslinya, cuma untuk testing aja
from transformers import (
    AutoTokenizer
)
from datasets import load_dataset

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
class Preprocessor:
    """
    Untuk preprocess sebelum masuk ke fine tuner
    input :
        preprocess_type : p0/p1/p2/p3
        tokenizer : huggingface tokenizer
        tokenizer_max_length : max length hasil tokenizer nantinya
        text_col : kolom text raw yang ingin dibersihkan
        label_col : kolom label yang ingin diprediksi
    out : 
        tokenized_inputs : huggingface dataset hasil pembersihan dan tokenizer
    """
    def __init__(self, 
                 preprocess_type:str , 
                 tokenizer, 
                 tokenizer_max_length:int, 
                 text_col:str,
                 label_col:str
        ):
        self.prepocess_type = preprocess_type
        self.tokenizer = tokenizer
        self.tokenizer_max_length = tokenizer_max_length
        self.text_col = text_col
        self.label_col = label_col
        if self.prepocess_type=='p01' or self.prepocess_type=='p03':
            #create stopword remover
            self.stop_factory = StopWordRemoverFactory()
            self.stopword_remover = self.stop_factory.create_stop_word_remover()
        if self.prepocess_type=='p02' or self.prepocess_type=='p03':
            # create stemmer
            self.factory = StemmerFactory()
            self.stemmer = self.factory.create_stemmer()
        
    def clean_repetitive(self, word):
        prev_char = None
        char_count=-1
        clean_word=''
        for c in word:
            if prev_char!=c:
                prev_char=c
                char_count=0
            else:
                char_count+=1
            if char_count<1:
                clean_word+=c
        #remove word if only 1 char left
        return clean_word if len(clean_word)>1 else ''
    def clean_text(self, text):
        #lower case
        text = text.lower()
        #clean text with tweet-preprocessor
        text = p.clean(text)
        #clean repetitive word
        text = " ".join([self.clean_repetitive(word) for word in text.split()])
        #convert slang word into dictionary
        text = " ".join([SLANG_WORDS[word] if word in SLANG_WORDS else word for word in text.split()])
        return text
    def stem(self, text):
        return self.stemmer.stem(text)
    def stopword_removal(self, text):
        return self.stopword_remover.remove(text)
    def preprocess_dataset(self, examples):
        inputs = examples[self.text_col]
        inputs = [self.clean_text(input) for input in inputs]
        if self.prepocess_type=='p02' or self.prepocess_type=='p04':
            inputs = [self.stopword_removal(input) for input in inputs]
        if self.prepocess_type=='p03' or self.prepocess_type=='p04':
            inputs = [self.stem(input) for input in inputs]
        targets =examples[self.label_col] 
        tokenized_inputs = self.tokenizer(
            inputs, text_target=targets, max_length=self.tokenizer_max_length, truncation=True
        )
        return tokenized_inputs
    

In [16]:
model_pretrained_checkpoint = "Wikidepia/IndoT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_pretrained_checkpoint)

In [19]:
max_length = 128
text_col = 'content'
label_col = 'quadruplet'
preprocessor = Preprocessor('p00', tokenizer, max_length, text_col, label_col)

In [21]:
#dataset
raw_dataset = load_dataset('csv', data_files='../Data/quadruplet/quadruplet_only.csv')
splitted_dataset = raw_dataset['train'].train_test_split(test_size=0.2, seed=42)
tokenized_dataset = splitted_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=splitted_dataset['train'].column_names)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-bf79bf034d0392b1/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 999.60it/s]
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-571d28bc462ba89a.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-24355355d1083472.arrow


In [23]:
tokenizer.decode(tokenized_dataset['test']['labels'][0])

'(pesananku, udah bayar tapi kenapa dibatalin, negative, website&apps);</s>'

# Fine tune

In [24]:
from transformers import (
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    AutoTokenizer
)
import torch

In [26]:
class FineTuner:
    def __init__(self, model, save_path, tokenizer, train_dataset, eval_dataset) -> None:
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.model = model.to(self.device)
        self.save_path = save_path
        self.tokenizer = tokenizer
        self.train_dataset = train_dataset
        self.eval_dataset = eval_dataset
        print(self.device)
    def fine_tune(self, arg):
        data_collator = DataCollatorForSeq2Seq(self.tokenizer, model=self.model)
        trainer = Seq2SeqTrainer(
            self.model,
            arg,
            train_dataset=self.train_dataset,
            eval_dataset=self.eval_dataset,
            data_collator=data_collator,
            tokenizer=self.tokenizer,
        )
        trainer.train()
        self.model.save_pretrained(self.save_path)

In [27]:
#constant
save_path = f'../models/test_fine_tuner'
model_pretrained_checkpoint = "Wikidepia/IndoT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_pretrained_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_pretrained_checkpoint)
#training argument
training_args = Seq2SeqTrainingArguments(
    save_path,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    resume_from_checkpoint=True,
    num_train_epochs=5,
    save_total_limit=2,
)
finetuner = FineTuner(model=model, save_path=save_path, tokenizer=tokenizer, 
                      train_dataset=tokenized_dataset['train'], eval_dataset=tokenized_dataset['test'])
finetuner.fine_tune(training_args)

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


cuda:0


  0%|          | 0/195 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                
 20%|██        | 39/195 [00:09<00:29,  5.34it/s]

{'eval_loss': 3.2513034343719482, 'eval_runtime': 0.4241, 'eval_samples_per_second': 181.551, 'eval_steps_per_second': 23.578, 'epoch': 1.0}


                                                
 40%|████      | 78/195 [00:40<00:22,  5.31it/s]

...

{'eval_loss': 2.185319185256958, 'eval_runtime': 0.3817, 'eval_samples_per_second': 201.746, 'eval_steps_per_second': 26.201, 'epoch': 4.0}


                                                 
100%|██████████| 195/195 [02:23<00:00,  5.48it/s]

{'eval_loss': 2.147050619125366, 'eval_runtime': 0.4064, 'eval_samples_per_second': 189.472, 'eval_steps_per_second': 24.607, 'epoch': 5.0}


100%|██████████| 195/195 [02:53<00:00,  1.12it/s]


{'train_runtime': 173.2776, 'train_samples_per_second': 8.887, 'train_steps_per_second': 1.125, 'train_loss': 2.9481980543870194, 'epoch': 5.0}


# Inference

In [46]:
from transformers import (
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
)
import torch
from tqdm import tqdm

In [47]:
class ModelInference:
    def __init__(self, batch_size, dataset, model, tokenizer) -> None:
        self.batch_size = batch_size
        self.model = model
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(self.device)
        self.datacollator = DataCollatorForSeq2Seq(tokenizer, model=model)
        self.dataset = dataset
        self.tokenizer = tokenizer
    def inference(self):
        print(len(self.dataset['input_ids']))
        inference_dataset = [self.dataset[i] for i in range(len(self.dataset['input_ids']))]
        inference_dataset = self.datacollator(inference_dataset)
        pred_text = []
        for i in tqdm(range(0, len(inference_dataset['input_ids']), self.batch_size)):
            generated_text = model.generate(inference_dataset['input_ids'][i:i+self.batch_size].to('cuda'), max_length=100)
            decoded_text = self.tokenizer.batch_decode(generated_text, skip_special_tokens=True)
            pred_text+=decoded_text
        return pred_text

In [48]:
#constant
save_path = f'../models/pt-indot5'
model = T5ForConditionalGeneration.from_pretrained(save_path).to('cuda')

In [50]:
model_inference = ModelInference(batch_size=8, dataset=tokenized_dataset['test'], model=model, tokenizer=tokenizer)
pred_text = model_inference.inference()

77


100%|██████████| 10/10 [00:08<00:00,  1.21it/s]


In [51]:
test_dataset = splitted_dataset['test']
test_dataset = test_dataset.add_column(f'model_prediction', pred_text)
test_dataset.to_csv('../Data/quadruplet/model_inference_300_data.csv')

Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-37557ea6cb933a33.arrow
Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 142.94ba/s]


37874

# Test pipeline dengan data sedikit
pakai data yg cuma 300an

In [1]:
from src.finetuner import FineTuner
from src.preprocessor import Preprocessor
from transformers import (
    T5ForConditionalGeneration,
    AutoTokenizer,
    Seq2SeqTrainingArguments
)
from datasets import load_dataset

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset

In [2]:
#constant
max_length = 128
text_col = 'content'
label_col = 'quadruplet'
preprocess_type = 'p00'
SAVE_PATH = f'../models/test_fine_tuner_with_300_data'
PRETRAINED_MODEL = "Wikidepia/IndoT5-base"
DATA_PATH = '../Data/quadruplet/quadruplet_only.csv'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
preprocessor = Preprocessor(preprocess_type, tokenizer, max_length, text_col, label_col)

In [4]:
raw_dataset = load_dataset('csv', data_files=DATA_PATH)
splitted_dataset = raw_dataset['train'].train_test_split(test_size=0.2, seed=42)
tokenized_dataset = splitted_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=splitted_dataset['train'].column_names)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-bf79bf034d0392b1/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 677.48it/s]
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-571d28bc462ba89a.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-24355355d1083472.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-6635831735983f73.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853b

In [5]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 308
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 77
    })
})

In [6]:
splitted_dataset['test']['content'][0]

'@tokopedia min aku udah bayar tapi kenapa dibatalin pesanan ku?'

In [7]:
tokenizer.decode(tokenized_dataset['test']['input_ids'][0])

'min aku sudah bayar tapi kenapa dibatalin pesanan ku?</s>'

## Model training

In [8]:
model = T5ForConditionalGeneration.from_pretrained(PRETRAINED_MODEL)

In [9]:
#training argument
training_args = Seq2SeqTrainingArguments(
    SAVE_PATH,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    resume_from_checkpoint=True,
    num_train_epochs=100,
    save_total_limit=2,
)

In [10]:
finetuner = FineTuner(model=model, save_path=SAVE_PATH, tokenizer=tokenizer, 
                      train_dataset=tokenized_dataset['train'], eval_dataset=tokenized_dataset['test'])

cuda:0


In [11]:
finetuner.fine_tune(training_args)

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/3900 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                  
  1%|          | 39/3900 [00:09<12:09,  5.29it/s]

{'eval_loss': 3.16196346282959, 'eval_runtime': 0.4602, 'eval_samples_per_second': 167.325, 'eval_steps_per_second': 21.73, 'epoch': 1.0}


                                                   
  2%|▏         | 78/3900 [00:40<10:59,  5.79it/s]

{'eval_loss': 2.395500659942627, 'eval_runtime': 0.4113, 'eval_samples_per_second': 187.215, 'eval_steps_per_second': 24.314, 'epoch': 2.0}


                                                   
  3%|▎         | 117/3900 [01:10<11:20,  5.56it/s]

...

                                                     
 56%|█████▌    | 2184/3900 [28:59<05:47,  4.94it/s]

{'eval_loss': 0.9796651601791382, 'eval_runtime': 0.4595, 'eval_samples_per_second': 167.561, 'eval_steps_per_second': 21.761, 'epoch': 56.0}


                                                     
 57%|█████▋    | 2223/3900 [29:32<05:32,  5.05it/s]

{'eval_loss': 0.9842897057533264, 'eval_runtime': 0.4498, 'eval_samples_per_second': 171.2, 'eval_steps_per_second': 22.234, 'epoch': 57.0}


KeyboardInterrupt: 

In [12]:
model.save_pretrained(SAVE_PATH)

## Inference

In [13]:
from src.inference import ModelInference
from src.preprocessor import Preprocessor
from transformers import (
    T5ForConditionalGeneration,
    AutoTokenizer,
)
from datasets import load_dataset

In [24]:
#constant
max_length = 128
text_col = 'content'
label_col = 'quadruplet'
preprocess_type = 'p00'
SAVE_PATH = f'../models/pt-indot5'
PRETRAINED_MODEL = "Wikidepia/IndoT5-base"
DATA_PATH = '../Data/quadruplet/quadruplet_only.csv'

In [25]:
model = T5ForConditionalGeneration.from_pretrained(SAVE_PATH).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

In [26]:
preprocessor = Preprocessor(preprocess_type, tokenizer, max_length, text_col, label_col)
#dataset
raw_dataset = load_dataset('csv', data_files=DATA_PATH)
splitted_dataset = raw_dataset['train'].train_test_split(test_size=0.2, seed=42)
tokenized_dataset = splitted_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=splitted_dataset['train'].column_names)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-bf79bf034d0392b1/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 497.37it/s]
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-571d28bc462ba89a.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-24355355d1083472.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-6635831735983f73.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853b

In [27]:
model_inference = ModelInference(batch_size=8, dataset=tokenized_dataset['test'], model=model, tokenizer=tokenizer)
pred_text = model_inference.inference()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 10/10 [00:08<00:00,  1.16it/s]


In [28]:
test_dataset = splitted_dataset['test']
test_dataset = test_dataset.add_column(f'{preprocess_type}_model_prediction', pred_text)
test_dataset.to_csv('../Data/quadruplet/model_inference_300_data.csv')

Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-bf79bf034d0392b1\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-37557ea6cb933a33.arrow
Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 139.76ba/s]


37878

## Evaluate

In [29]:
import pandas as pd
from src.evaluator import Evaluator
from src.postprocessor import PostProcessor

In [36]:
df = pd.read_csv('../Data/quadruplet/model_inference_300_data.csv')
df.head(5)

,original_id,content,clean_tweet,final_sentiment,labels,quadruplet,spam,sentiment_label,p00_model_prediction
0,1.640000e+18,@tokopedia min aku udah bayar tapi kenapa diba...,min aku udah bayar tapi kenapa dibatalin pesan...,negative,payment; produk;,"(pesananku, udah bayar tapi kenapa dibatalin, ...",NaN,sentiment,"(pesananku, sudah bayar tapi kenapa dibatalin,..."
1,1.640000e+18,Seperti biasa menghibungi call center @sicepat...,Seperti biasa menghibungi call center hanya be...,negative,delivery; produk;,"(call center, hanya berbelit belit, negative, ...",NaN,sentiment,"(cal center, hanya berbelit belit, negative, c..."
2,1.640000e+18,@tanyakanrl alfa kalo jumat-minggu ada promo t...,alfa kalo jumat-minggu ada promo tuh jd murah ...,neutral,price; produk;,"(alfa, kalo jumat-minggu ada promo, positive, ...",NaN,sentiment,"(alfa, kalau jumat-mingu ada promo, positive, ..."
3,1.640000e+18,@tokopedia apakah tokopedia care sedang gangguan?,apakah tokopedia care sedang gangguan?,negative,website&apps; produk;,"(tokopedia, gangguan, negative, website&apps);",NaN,sentiment,"(tokopedia care, sedang ganguan, neutral, webs..."
4,1.640000e+18,"Beli token listrik lewat tokped error, direjec...","Beli token listrik lewat tokped error, direjec...",negative,website&apps; payment; produk;,"(token listrik, lewat tokped error, negative, ...",NaN,sentiment,"(token listrik, lewat tokped eror, negative, w..."


In [31]:
postprocessor = PostProcessor(use_postprocess=False)
evaluator = Evaluator(task_type='quadruplet', postprocessor=postprocessor)

In [32]:
raw_scores, all_labels, all_preds = evaluator.evaluate(pred_seqs=df['p00_model_prediction'],
                   gold_seqs=df['quadruplet'])

100%|██████████| 77/77 [00:00<00:00, 76913.89it/s]


In [33]:
raw_scores

{'precision': 0.3333333333333333,
 'recall': 0.3333333333333333,
 'f1': 0.3333333333333333}

In [35]:
all_labels

[[('pesananku',
   'udah bayar tapi kenapa dibatalin',
   'negative',
   'website&apps')],
 [('call center', 'hanya berbelit belit', 'negative', 'customerservice'),
  ('transaksi', 'tdk bisa di cancel', 'negative', 'delivery')],
 [('alfa', 'kalo jumat-minggu ada promo', 'positive', 'price')],
 [('tokopedia', 'gangguan', 'negative', 'website&apps')],
 [('token listrik', 'lewat tokped error', 'negative', 'website&apps')],
 [('default paymentnya', 'tanpa ada konfirmasi ulang', 'negative', 'payment')],
 [('payday', 'bulan ini gak dapet apa-apa', 'neutral', 'product')],
 [('tiktok shop', 'dapat harga plg murah', 'positive', 'price')],
 [('tokopedia', 'makin ke sini makin pelit aja', 'negative', 'price')],
 [('ghd', 'lebi lancar nyatok rambutnya', 'positive', 'product'),
  ('hairbeauron', 'lebih tersendat', 'negative', 'product'),
  ('catokan mahal', 'gak bau gosong', 'positive', 'product')],
 [('tokopedia', 'ongkir kelewat mahal', 'negative', 'delivery'),
  ('tokopedia', 'sistem gak aman', 

In [34]:
all_preds

[[('pesananku',
   'sudah bayar tapi kenapa dibatalin',
   'negative',
   'website&apps')],
 [('cal center', 'hanya berbelit belit', 'negative', 'customerservice'),
  ('transaksi',
   'tidak bisa di cancel karena sudah status menungu',
   'negative',
   'delivery')],
 [('alfa', 'kalau jumat-mingu ada promo', 'positive', 'price')],
 [('tokopedia care', 'sedang ganguan', 'neutral', 'website&apps')],
 [('token listrik', 'lewat tokped eror', 'negative', 'website&apps')],
 [('default paymentnya', 'tanpa ada konfirmasi ulang', 'negative', 'payment')],
 [('payday', 'bulan ini gak dapat apa-apa', 'neutral', 'product')],
 [('tokopedia', 'dapat harga paling murah', 'positive', 'price'),
  ('shopee', 'dapat harga paling murah', 'positive', 'price')],
 [('tokopedia', 'makin ke sini makin pelit saja', 'negative', 'price')],
 [('ghd', 'lebi lancar nyatok rambutnya', 'positive', 'product'),
  ('hairbeauron', 'lebih tersendat', 'negative', 'product')],
 [('tokopedia', 'ongkos kirim kelewat mahal', 'ne